# 基于LDA主题模型和情感词典的Twitter推文主题提取及情感分析

## 工作环境：(jupyter notebook)


python 3.7.5

pandas 1.0.1 (处理表格数据)

wordcloud 1.8.1 (词云)

stylecloud 0.5.1 (词云)

nltk 3.5 (内含各种停用词，以及词干提取，词形还原和分词的方法)

gensim 3.8.3 (内含LDA模型)

pyLDAvis 3.2.2 (LDA模型可视化工具)

numpy 1.17.4 (数学工具)

matplotlib 3.1.1 (数据可视化绘图工具)

seaborn 0.11.0

Pillow 7.1.2


## 概念说明
 
 
#### 什么是LDA？

> LDA（隐含狄利克雷分布）**将文档集中的每篇文档的主题以概率分布的形式给出**



#### 什么是TF-IDF？

> TF: 词频(term frequency) （对于某个文本，单词在某个文本中出现越多次越重要）

> IDF: 逆文档频率(inverse document frequency) （单词在语料库（所有文本）中出现越少次越重要）

> TF-IDF是一种衡量某一篇文档中某个词对该篇文档重要程度的计算方法。通过TF-IDF公式，我们可以计算出这个词对于表现这篇文档主题而言贡献如何。TFIDF的主要思想是：如果某个词或短语在一篇文章中出现的频率TF高，并且在其他文章中很少出现，则认为此词或者短语具有很好的类别区分能力，适合用来分类或提取关键词。




#### 什么是情感词典？




## 流程
1. 数据获取:

使用python爬虫爬取twitter数据,时间线(2020.3.12-2021.1.12)

2. 数据清洗:

    ① 清除存在推文或发布时间缺失的数据条目
    
    ② 将数据按照发布时间前后顺序排序
    
    ③ 数据分布初步分析（随发布时间）

3. 文本预处理:

    ① Twitter_data['content'] 初始数据(matrix size:)
    
    ② Processed_data 去除了标点、@、短词(matrix size:)
    
    ③ data_words 去除了nltk停用词词典中的停用词、自定义的禁用词列表（如搜索关键词）(matrix size:)


4. **情感分析:**

    ① 根据TF-IDF算法对推文进行关键词提取，进行词云展示（总体、逐月）
    
    ② 使用LDA主题模型进行训练，提取主题分布
    
        1. 构建LDA模型，进行训练

        2. 计算模型困惑度(Perplexity)和模型一致性得分(Coherence Score)，其中困惑度越低越好，一致性得分越高越好

        3. 为每篇推文分配对应占主导地位的主题（概率较高的）

        4. 调整主题选取数或训练迭代次数等其他参数，迭代调优
        
    ③ 根据情感词典SentiWordNet3.0进行文本得分计算，每篇推文会计算出一个得分。其情感越积极，则得分越高；反之情感越消极，则得分越低
 
 

5. 结果分析



##### 优点：
- 相比于使用机器学习和深度学习的方法，使用情感词典计算文本得分，无需经过模型的训练和调参，过程更为直观，也更适合于像twitter推文这种短文本的分析（短文本通常有很多逻辑不完整且不符合语法规则的句子）

- 



##### 局限性：
- 情感词典方面，单词得分直接由相关单词得出，并未考虑程度副词的影响

- 

## 一、数据清洗

In [ ]:
# 导入相关包
import pandas as pd
import os
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
from gensim.models.tfidfmodel import TfidfModel
from gensim.models.ldamodel import LdaModel
from gensim.models.coherencemodel import CoherenceModel


import nltk
from nltk.corpus import stopwords

import re
import string

import seaborn as sns

from pprint import pprint

from wordcloud import WordCloud

from PIL import Image

import pyLDAvis.gensim

import stylecloud


## 初始化参数设置
- 全局选取关键词Top N
- 逐月选取关键词Top N
- 主题数选取
- 训练迭代次数

In [ ]:
# 全局选取关键词Top N
GLOBAL_TOP_N = 20

# 逐月选取关键词Top N
PER_MONTH_TOP_N = 10

# 主题数选取
NUM_TOPICS = 4

# 训练迭代次数
ITERATIONS = 50

In [ ]:
# 指定当前工作路径
# print(os.getcwd())
os.chdir(os.getcwd())

# 从csv文件中读入数据
Twitter_data = pd.read_excel('./twitter_covid.xlsx')

# 初始数据部分展示
Twitter_data.head()

In [ ]:
# 规范化数据 列名称
Twitter_data.rename(
    columns={'m_content': 'content', 'g_publish_time': 'publish_time', 'm_content_id': 'content_id'},
    inplace=True
)

In [ ]:
# 爬取初始数据存在空行，需要进行处理
# 可以看到有11744条非空推文，却有11841条发布日期，所以数据肯定存在部分缺失
Twitter_data.info()

In [ ]:
# 清除缺失推文或缺失发布时间的数据
Twitter_data.dropna(axis=0, how='any', inplace=True)

In [ ]:
# 清除了推文为空的数据项
Twitter_data.info()

In [ ]:
# 将发布时间转换成python中的datetime格式,便于后续操作,并将发布时间统一为 year-month-date 的格式

# PS:不使用pd.to_datetime是因为对于某一条数据，我们只关心其发布的日期，在一天中的具体时间并不关心，
# 而pd.datetime会带上 hour-min-sec ，不利于后续操作
Twitter_data['publish_time'] = Twitter_data['publish_time'].map \
(lambda x: datetime.date((int)(x.split('-')[0]), (int)(x.split('-')[1]), (int)(x.split('-')[2].split(' ')[0])))


In [ ]:
# 所有数据按照发布时间排序
Twitter_data = Twitter_data.sort_values(by='publish_time')
Twitter_data.reset_index(inplace=True)
del Twitter_data['index']

In [ ]:
# 观察到不在预期时间范围内的数据，进行清除
Twitter_data.head()

In [ ]:
#设定起始、截止日期
start_date = datetime.date(2020, 3, 12)
end_date = datetime.date(2021, 1, 12)


for index, date in enumerate(Twitter_data['publish_time']):
    # 在起始日期前或截止日期后,为不符合要求的数据
    if date.__sub__(start_date).days<0 or date.__sub__(end_date).days>0:
        Twitter_data.drop(index=index, inplace=True)
        
Twitter_data = Twitter_data.reset_index()
del Twitter_data['index']

In [ ]:
# 可以观察到数据开始和结束都已经符合预期
Twitter_data.head()

In [ ]:
Twitter_data.tail()

### 观察原数据在2020.3.12-2021.1.12的分布情况

In [ ]:
def format_month_or_day(month_or_day):
    if int(month_or_day)>0 and int(month_or_day)<10:
        return '0' + str(month_or_day)
    else:
        return month_or_day

In [ ]:
# 将publish_time进行拆分便于后续统计
Twitter_data['year'] = Twitter_data['publish_time'].map(lambda x: x.year)
Twitter_data['month'] = Twitter_data['publish_time'].map(lambda x: x.month)
Twitter_data['date'] = Twitter_data['publish_time'].map(lambda x: format_month_or_day(x.day))
Twitter_data['year_month'] = Twitter_data['publish_time'].map(lambda x: str(x.year) + '-' + str(format_month_or_day(x.month)))
Twitter_data['times'] = Twitter_data['publish_time'].map(lambda x: 1)

In [ ]:
Twitter_data.head()

In [ ]:
# 对于某一天的数据频次统计如下,times为频次
day_data = Twitter_data.groupby(['publish_time'], as_index=False).agg({'times': 'sum'})

In [ ]:
day_data

In [ ]:
# 对于某个月的数据频次统计如下，times为频次
month_data = Twitter_data.groupby(['year_month', 'year', 'month'], as_index=False).agg({'times': 'sum'})
month_data['day'] = 1
month_data['publish_time'] = pd.to_datetime(month_data[['year', 'month', 'day']]) 

In [ ]:
month_data[['year_month', 'times']]

In [ ]:
# 数据量随日期分布的折线图，可以看到某些日期的数据量较少，总体来说相对均匀
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
fig.set_size_inches(16,5)

plt.plot(day_data['publish_time'], day_data['times'], linewidth=1.3, label='Daily', color='orange')
ax.set(xlabel='count',title='Distribution of tweets by date')
plt.legend()
plt.show()

In [ ]:
# 数据量随月份分布的柱状图，可以看到2020-3和2021-1的数据量较少，其他月份数据量都较均匀
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
fig.set_size_inches(16,5)
rects = plt.bar \
(range(len(month_data['month'])), month_data['times'], tick_label=month_data['year_month'], color='rgb')

# 打数据标注
for rect in rects:  #rects 是三根柱子的集合
    height = rect.get_height()
    plt.text(rect.get_x() + rect.get_width() / 2, height, str(height), size=13, ha='center', va='bottom')

plt.title('Distribution of tweets by year-month')
plt.show()

In [ ]:
# plt.figure(figsize=(10,10))
# plt.hist(Twitter_data['publish_time'], density=True, bins='auto',align='left')

In [ ]:
# 数据量随月份分布的饼图
fig = plt.figure()
fig.set_size_inches(8,8)

plt.pie(month_data['times'],labels=month_data['year_month'],autopct='%1.1f%%',shadow=False,startangle=150)
plt.title('Distribution of tweets by year-month')
plt.show()

#### 数据分布情况：2020-3和2021-1的数据量较少，其他月份数据量都较均匀

## 二、文本预处理

In [ ]:
# print(string.punctuation)

def text_preprocess(data):
    # 去除@user
    processed_data = data.map(lambda x: re.sub(r'@[\w]*', '', x))

    # 去除网址
    processed_data = processed_data.map(lambda x: re.sub(r'http.*\..* ', '', x))

    # 去除标点符号
    processed_data = processed_data.map(lambda x: re.sub(r'[^a-zA-Z ]', '', x))

    # 删除长度太短的词（通常无意义）
    processed_data = processed_data.map(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

    # 分词
    processed_data = processed_data.map(lambda x: nltk.WordPunctTokenizer().tokenize(x))

    return processed_data

In [ ]:
# 去除标点、@、短词，进行分词
Processed_data = text_preprocess(Twitter_data['content'])

In [ ]:
# 经过预处理后的单词矩阵
Processed_data.head()

In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')

In [ ]:
# 查看停用词词典
stopwords.words('english')

In [ ]:
# 去除停用词
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]


# 去除停用词
data_words = remove_stopwords(Processed_data)


In [ ]:
# 去除停用词后的单词矩阵
data_words

In [ ]:
# 去除搜索关键词



In [ ]:
# 导入下面三种词干提取器进行对比
import nltk.stem.porter as pt
import nltk.stem.lancaster as lc
import nltk.stem.snowball as sb

# 导入nltk.stem用来词型还原
import nltk.stem as ns

def word_stem(text_matrix):

    print("----------词干提取-------------")
    # 在名词和动词中，除了与数和时态有关的成分以外的核心成分。
    # 词干并不一定是合法的单词

    pt_stemmer = pt.PorterStemmer()  # 波特词干提取器
    lc_stemmer = lc.LancasterStemmer()   # 兰卡斯词干提取器
    sb_stemmer = sb.SnowballStemmer("english")# 思诺博词干提取器

    print("%8s %8s %8s %8s" % ('word','pt_stem','lc_stem','sb_stem'))
    for text in text_matrix:
        for word in text:
            pt_stem = pt_stemmer.stem(word)
            lc_stem = lc_stemmer.stem(word)
            sb_stem = sb_stemmer.stem(word)
            print("%8s %8s %8s %8s" % (word,pt_stem,lc_stem,sb_stem))




In [ ]:
# 展示一下词干提取的结果，但实际并没有使用词干提取，因为粒度过粗
word_stem(data_words)

immunity    immun    immun    immun
 bidding      bid      bid      bid
 vaccine   vaccin   vaccin   vaccin
    done     done      don     done
guyreporter guyreport guyreport guyreport
   wants     want     want     want
   argue     argu     argu     argu
   trump    trump    trump    trump
    hope     hope      hop     hope
 working     work     work     work
   hates     hate      hat     hate
   trump    trump    trump    trump
something   someth   someth   someth
    cure     cure      cur     cure
   covid    covid    covid    covid
   covid    covid    covid    covid
 vaccine   vaccin   vaccin   vaccin
  donald   donald   donald   donald
   trump    trump    trump    trump
    call     call      cal     call
    fake     fake      fak     fake
    news     news      new     news


In [ ]:
# 词型还原：复数名词->单数名词 ；分词->动词原型
# 单词原型一定是合法的单词

def word_lemma(text_matrix): 
    print("----------词型还原器---------------")
    
    print("%8s %8s %8s" % ('word','n_lemma','v_lemma'))
    lemmatizer = ns.WordNetLemmatizer()
    for index_txt, text in enumerate(text_matrix):
        for index_word, word in enumerate(text):
            # 将名词还原为单数形式
            n_lemma = lemmatizer.lemmatize(word, pos='n')
            # 将动词还原为原型形式
            v_lemma = lemmatizer.lemmatize(word, pos='v')
            print('%8s %8s %8s' % (word, n_lemma, v_lemma))
            
            if(len(n_lemma)<len(v_lemma)):
                text_matrix[index_txt][index_word] = n_lemma
            else:
                text_matrix[index_txt][index_word] = v_lemma
                
    return text_matrix
            

In [ ]:
# 实际使用了粒度更细的词形还原
data_words = word_lemma(data_words)

In [ ]:
# 经过词形还原后的单词矩阵
data_words

In [ ]:
# 去除搜索关键词相关的单词
ban_words_list = []
with open('./ban_words_list.txt') as f:
    for line in f.readlines():
        ban_words_list.append(line.strip('\n'))

In [ ]:
ban_words_list

In [ ]:
def remove_search_words(words_matrix, ban_words_list):
    words_processed = []
    for text in words_matrix:
        words_text = []
        for word in text:
            if word not in ban_words_list:
                words_text.append(word)
                
        words_processed.append(words_text)
        
    return words_processed
    

In [ ]:
data_words = remove_search_words(data_words, ban_words_list)

In [ ]:
data_words

## TF-IDF（关键词权重）与LDA（词分布）的区别

> 使用TF-IDF进行关键词提取的局限性在于: 在某些场景，基于文档本身的关键词提取还不是非常足够，有些关键词并不一定会显示地出现在文档当中，如一篇讲动物生存环境的科普文，通篇介绍狮子老虎等，但是文中并没有显示地出现动物二字。

> 而LDA主题模型认为在词与文档之间没有直接的联系，它们应当还有一个维度将它们串联起来，主题模型将这个维度称为主题。每个文档都应该对应着一个或多个的主题，而每个主题都会有对应的词分布，通过主题，就可以得到每个文档的词分布。

## 词云分析

基于TF-IDF算法生成词云


### 词袋模型(bag of words)
- 将每一篇文档视为一个词频向量，从而将文本信息转化为了易于建模的数字信息
- 词袋方法没有考虑词与词之间的顺序，这简化了问题的复杂性，同时也为模型的改进提供了契机

In [ ]:
# 构造词典（将单词词典化，编号和单词具有一一对应关系）
dictionary = corpora.Dictionary(data_words)

texts = data_words

# 统计词频，构造词频矩阵（词袋）
corpus = [dictionary.doc2bow(text) for text in texts]


In [ ]:
# 构造TF-IDF模型
tf_idf_model = TfidfModel(corpus, normalize=True)
word_tf_idf = list(tf_idf_model[corpus])

In [ ]:
# 词典
dictionary.token2id

In [ ]:
# 词频（每一行代表一篇推文，某一行中的某个元组对（x,y）代表编号为x的单词在该推文中出现了y次）
corpus

In [ ]:
# 每个句子对应的词袋 (前面是单词，后面是词频)
# 词频矩阵反向化（id->单词）

id_words = [[(dictionary[id], count) for id, count in line] for line in corpus]
print(data_words[1])
print(id_words[1])

In [ ]:
# TF-IDF得分，每篇推文中每个单词都有对应的TF_IDF得分，该得分已经进行了标准化，可以视作一个单词的权重
word_tf_idf

In [ ]:
# 基于TF-IDF算法提取关键词
def TF_IDF_get_keywords(words_matrix, top_n=3):
    # 构建词袋模型

    # 构造词典（将单词词典化，编号和单词具有一一对应关系）
    dictionary = corpora.Dictionary(words_matrix)

    texts = words_matrix

    # 统计词频，构造词频矩阵（词袋）
    corpus = [dictionary.doc2bow(text) for text in texts]
    
    tf_idf_model = TfidfModel(corpus, normalize=True)
    word_tf_idf = list(tf_idf_model[corpus])
    
    key_words_matrix = []
    
    for index_txt, text in enumerate(word_tf_idf):
        print('tweet %d \'s keywords: ' % index_txt)
        text.sort(key=lambda x: x[1], reverse=True)
        key_words_code = [word[0] for word in text[:top_n+3]]
        key_words = []
#         print(key_words_code)
        for code in key_words_code:
            if dictionary[code] not in ban_words_list:
                key_words.append(dictionary[code])
            if len(key_words)>=3:
                break
        
        print(key_words)
        print()
            
        key_words_matrix.append(key_words)
    
    return key_words_matrix
        
    
    

In [ ]:
# 提取每篇推文Top3的TF-IDF得分的关键词
key_words_global = TF_IDF_get_keywords(data_words)

In [ ]:
key_words_global

In [ ]:
# 提取高频关键词
def get_highest_frequency_words(words, top_n=10):
    frequency = {}
    for word in words:
        if word in frequency:
            frequency[word] += 1
        else:
            frequency[word] = 1
    
#     print(frequency)
    top_words = sorted(frequency.items(), key=lambda x: x[1],reverse=True)[:top_n]
    
#     print(top_words)
    top_words_dict = {}
    for word in top_words:
        top_words_dict[word[0]] = word[1]
        
        
    return top_words_dict

In [ ]:
# 提取每个月出现频率最高的关键词
key_words_global_general = {}

key_words = []
for text in key_words_global:
    for word in text:
        key_words.append(word)
    
# print(key_words)

key_words_global_general = get_highest_frequency_words(key_words, top_n=GLOBAL_TOP_N)

# 将Top20关键词输出至./keywords/global/Top20_keywords_global.txt文件中
if not os.path.exists('./key_words/global'):
    os.makedirs('./key_words/global')
if not os.path.exists('./key_words/global/Top20_keywords_global.txt'):

    with open('./key_words/global/Top20_keywords_global.txt', 'w') as f:
        for key_word in key_words_global_general:
            f.write(key_word+' '+str(key_words_global_general[key_word])+'\n')
    

In [ ]:
key_words_global_general

In [ ]:
# 生成全局词云

# 连接全局关键词矩阵中所有单词
global_string =  ' '.join([' '.join(word) for word in key_words_global])

# # Create a WordCloud object
# wordcloud = WordCloud \
# (height=400, width=800, background_color="white", max_words=100, contour_width=3, contour_color='steelblue')

# 
# wordcloud.generate(global_string)
# wordcloud.to_image()
# if not os.path.exists('./word_cloud/global/wordcloud_global.png'):
#     wordcloud.to_file('./word_cloud/global/wordcloud_global.png')


if not os.path.exists('./word_cloud/global'):
    os.makedirs('./word_cloud/global')

# 生成词云    
    
if not os.path.exists('./word_cloud/global/wordcloud_global.png'):
    
    stylecloud.gen_stylecloud( 
        text=global_string, 
        icon_name="fab fa-twitter", # 使用推特图标蒙版
        gradient='horizontal', # 渐变色方向选了垂直方向
        max_words=200,
        output_name='./word_cloud/global/wordcloud_global.png',
    )

# 词云可视化

img = Image.open('./word_cloud/global/wordcloud_global.png')
    
fig = plt.figure(figsize=(12,8))
plt.imshow(img)
plt.axis('off')
plt.title('global wordcloud', fontsize='xx-large',fontweight='heavy')
plt.show()

In [ ]:
month_data['times']

In [ ]:
# 逐月分割关键词矩阵
'''
input: 单词矩阵（分词后的单词 / 提取后的关键词矩阵）, shape:
       逐月数据条数（每个月有多少条数据，前面已经统计过了）, 
       具体月份（这里使用year-month）
       
output: 逐月分割后的单词矩阵（增加了一个维度） 
'''

def words_separate_by_month(words_matrix, times_by_month, year_month):
    '''
    @description: 
    @param 
    @return: 
    '''
    
    key_words_by_month = {}
    
    global_index = 0
    
    for index, times in enumerate(times_by_month):
        key_words_per_month = []
        for i in range(times):
            key_words_per_month.append(words_matrix[global_index])
            global_index += 1
            
        key_words_by_month[year_month[index]] = key_words_per_month
    
    
    return key_words_by_month

In [ ]:
# 生成逐月词云
key_words_by_month = words_separate_by_month(key_words_global, month_data['times'], month_data['year_month'])

In [ ]:
# 2020年11月的前5条数据
key_words_by_month['2020-11'][:5]

In [ ]:
# 提取每个月出现频率最高的关键词
key_words_by_month_general = {}

for year_month in key_words_by_month.keys():
    key_words_per_month = []
    for text in key_words_by_month[year_month]:
        for word in text:
            key_words_per_month.append(word)
    
#     print(key_words_per_month)
    
    key_words_by_month_general[year_month] = get_highest_frequency_words \
    (key_words_per_month, top_n=PER_MONTH_TOP_N)
    
    if not os.path.exists('./key_words/per_month'):
        os.makedirs('./key_words/per_month')
    if not os.path.exists('./key_words/per_month/Top10_keywords_'+year_month+'.txt'):

        with open('./key_words/per_month/Top10_keywords_'+year_month+'.txt', 'w') as f:
            for key_word in key_words_by_month_general[year_month]:
                f.write(key_word+' '+str(key_words_by_month_general[year_month][key_word])+'\n')
    

In [ ]:
# 每个月的top10关键词
pprint(key_words_by_month_general)

In [ ]:
# 生成逐月词云

for year_month in key_words_by_month.keys():
    
    # 连接逐月关键词矩阵中所有单词
    per_month_string = ' '.join([' '.join(x) for x in key_words_by_month[year_month]])

#     print(len(per_month_string))
    # Create a WordCloud object
    wordcloud = WordCloud \
    (height=400, width=800, background_color="white", max_words=100, contour_width=3, contour_color='steelblue')

    # 生成词云
    wordcloud.generate(per_month_string)

    if not os.path.exists('./word_cloud/per_month'):
        os.makedirs('./word_cloud/per_month')
    
    if not os.path.exists('./word_cloud/per_month/wordcloud_'+year_month+'.png'):
        wordcloud.to_file('./word_cloud/per_month/wordcloud_'+year_month+'.png')
    
    
    img = Image.open('./word_cloud/per_month/wordcloud_'+year_month+'.png')
    
    fig = plt.figure(figsize=(12,8))
    plt.imshow(img)
    plt.axis('off')
    plt.title('word cloud of %s' % year_month, fontsize='xx-large',fontweight='heavy',color='white')
    plt.show()

## 什么是LDA主题模型?

在LDA中，所有的文档共有同样的话题集，但是每个文档以不同的比例展示对应的话题。LDA的主要目标是自动发现一个文档集合中的话题。这些文档本身是可以观测到的，而话题的结构——话题、每个文档的话题分布和每个文档的每个词的话题赋值——是隐藏的（可称为hidden structure）。话题建模的核心计算问题就是使用观测到的文档来推断隐藏话题结构。这也可以看作是生成（generative）过程的逆过程——什么样的隐藏结构可以产生观测到的文档集合？


### LDA模型训练

默认设置有5个topic

期望topic在四个象限分布越均匀越好，代表覆盖得越全面

若topic之间隔的太远，则增大topic的数量

若不同topic之间有较大的重叠，则减少topic的数量


In [ ]:
# 读取或创建lda模型
if not os.path.exists('./lda_model'):
    os.makedirs('./lda_model')

if os.path.exists('./lda_model/lda.model'):
    lda_model = LdaModel.load('./lda_model/lda.model')
else:

    # 设定的主题数目
    num_topics = NUM_TOPICS
    iterations = ITERATIONS

    # 构建LDA模型
    lda_model = gensim.models.LdaMulticore(
        corpus=corpus,
        id2word=dictionary,
        num_topics=num_topics
    )

    lda_model.save('./lda_model/lda.model')

    
topic_list = lda_model.print_topics(num_words=10)
# doc_lda = lda_model[corpus]

In [ ]:
# 主题分布
topic_list

In [ ]:
# 模型困惑度得分
# 得分越低说明模型对文本越不困惑
print('Perplexity: ', lda_model.log_perplexity(corpus)) # a measure of how good the model is. lower the better.

# Compute Coherence Score
# 模型一致性得分
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

In [ ]:
# 格式化输出所有主题构成（概率分布）
if not os.path.exists('./lda_topics'):
    os.makedirs('./lda_topics')
    
if not os.path.exists('./lda_topics/topics.txt'):
    with open('./lda_topics/topics.txt', 'w') as f:
        for topic in topic_list:
            f.write('topic %d:\n' % topic[0])
            for word in topic[1].split('+'):
#                 print(word)
                f.write('  {}  {}\n'.format(word.split('*')[1], word.split('*')[0]))
                
            f.write('\n')
    
    

In [ ]:
# 每个文档都包含多个主题。但是，通常只有一个主题是主导的。
# 下面的代码为每个句子提取该主要主题，并在格式正确的输出中显示该主题和关键字的权重。
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()
    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0: # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append\
                (pd.Series([int(topic_num),round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    # print(contents)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    # print(sent_topics_df)
    return(sent_topics_df)

df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_words)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Topic_Keywords', 'Text']



df_dominant_topic.to_csv('./lda_topics/dominant_topics.csv',index=False)

In [ ]:
# 使用LDA对每篇推文的主要主题筛选
df_dominant_topic

## LDA可视化
- 每个主题的意义（右边的单词为某个主题下常出现的单词）
- 每个主题在总语料库中的比重
- 主题之间的关联


In [ ]:
# LDA模型可视化

vis_data = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis_data)

In [ ]:
pyLDAvis.show(vis_data)

### 情感词典计算文本得分

In [ ]:
SentiWordNet = pd.read_csv('./SentiWordNet3.0.0.csv')

In [ ]:
SentiWordNet['attr'].value_counts()